#  French Trot Horse Racing Prediction Using Artificial Neural Networks

**CIS*6020: Artificial Intelligence**

University of Guelph

Submitted By: Santosh Kumar Satapathy
Under the Guidance of: Dr. Neil Bruce



## Importing Necessary Libraries

In [8]:
# Basic Libraries
import pandas as pd
import numpy as np
from tqdm import tqdm

# Import Google Drive library to import data from Google Drive as mentioned in the ReadMe Instructions
from google.colab import drive

# Sklearn Libraries Used
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

# Tensorflow Libraries Used
import tensorflow as tf
from tensorflow import keras
from keras import backend as K

## Mount Drive and Read Data

In [9]:
drive.mount('/content/drive')

# Reminder: The dataset parquet file is supposed to be located at My Drive path in Google Drive
# Else, the path below needs to be updated before running:

data = pd.read_parquet('/content/drive/My Drive/trots_2013-2022.parquet', engine='pyarrow')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data PreProcessing and Data Engineering

## Finish Position and Finish Position 2

In [10]:
# Remove unnecessary space characters
data['FinishPosition'] = data['FinishPosition'].str.replace(' ','')

# Feature Engineering: New Feature that captures whether a horse was able to finish the race or got disqualified
data['FinishPosition2'] = np.where(data['FinishPosition'].isin(['BS','UN','PU','DQ','FL','NP','UR','WC']),1,0)

# In the original feature, assign the last finish position in case of disqualified horses
data['FinishPosition'] = np.where(data['FinishPosition']=='BS',19,data['FinishPosition'])
data['FinishPosition'] = np.where(data['FinishPosition']=='UN',20,data['FinishPosition'])
data['FinishPosition'] = np.where(data['FinishPosition']=='PU',21,data['FinishPosition'])
data['FinishPosition'] = np.where(data['FinishPosition']=='DQ',22,data['FinishPosition'])
data['FinishPosition'] = np.where(data['FinishPosition']=='FL',23,data['FinishPosition'])
data['FinishPosition'] = np.where(data['FinishPosition']=='NP',24,data['FinishPosition'])
data['FinishPosition'] = np.where(data['FinishPosition']=='UR',25,data['FinishPosition'])
data['FinishPosition'] = np.where(data['FinishPosition']=='WC',26,data['FinishPosition'])

# Convert to Numerical Feature
data['FinishPosition'] = data['FinishPosition'].astype(int)

## Beaten Margin

In [11]:
# In case of Beaten Margin of disqualified horses, we are changing the margin from default 999 value to the max value of beaten margin of that race + the standard deviation
calc = data[data['BeatenMargin']!=999][['BeatenMargin','RaceID']].groupby(by=['RaceID'],as_index=False).max().join(data[data['BeatenMargin']!=999][['BeatenMargin','RaceID']].groupby(by=['RaceID'],as_index=False).std(),rsuffix='2')
del calc['RaceID2']
data = data.merge(calc,how='left',on=['RaceID'])
del calc
data['BeatenMargin2'] = data['BeatenMargin2'].fillna(0)

# Assign new value and clean dataset
data['BeatenMargin_x'] = np.where(data['BeatenMargin_x']==999,data['BeatenMargin_y']+data['BeatenMargin2'],data['BeatenMargin_x'])
del data['BeatenMargin2']
del data['BeatenMargin_y']
data['BeatenMargin'] = data['BeatenMargin_x']
del data['BeatenMargin_x']

## Imputing NULL Values

In [12]:
# This loop checks for Null OR Garbage Values in the dataset and prints these features and values
for i in data.columns:
    if ((len(data[(data[i].isna())|(data[i].isnull())|(data[i]==None)|(data[i]=='None')|(data[i]=='')].index)/len(data.index)*100)>0):
        print(i+" Null or Garbage Value Count :  "+str(len(data[(data[i].isna())|(data[i].isnull())|(data[i]==None)|(data[i]=='None')|(data[i]=='')].index)))
        data[i] = np.where((data[i].isna())|(data[i].isnull())|(data[i]==None)|(data[i]=='None')|(data[i]==''),'NA',data[i])

AgeRestriction Null or Garbage Value Count :  633
ClassRestriction Null or Garbage Value Count :  29588
HandicapType Null or Garbage Value Count :  1042792
SexRestriction Null or Garbage Value Count :  888663


## Defining Features

In [13]:
# Keeping the data sorted by Race Time as we know that we will be feeding the dataset in a chronological order while training so it would be easy to feed later
data.sort_values(by=['RaceStartTime','FinishPosition'],ascending=[True,False])
data = data.reset_index(drop=True)

# Defining our Race Data Related Features
data_col = ['AgeRestriction','Barrier','ClassRestriction','CourseIndicator','DamID','Distance','FoalingCountry','FoalingDate','FrontShoes','GoingAbbrev',
            'GoingID','HandicapDistance','HandicapType','HindShoes','HorseAge','HorseID','JockeyID','RaceGroup','RacePrizemoney','RaceStartTime','RacingSubType',
            'Saddlecloth','SireID','StartType','StartingLine','Surface','TrackID','TrainerID','WeightCarried','WetnessScale','RaceID','SexRestriction','Gender']

# Defining our Performance Data Related Features. We will be using Beaten Margin from all these features as our Label.
perf_col = ['BeatenMargin','FinishPosition','FinishPosition2','Disqualified', 'PIRPosition', 'Prizemoney', 'RaceOverallTime', 'PriceSP', 'NoFrontCover', 'PositionInRunning',
               'WideOffRail']

# Alter the names of Performance Features for easy recognition
for i in data[perf_col].columns:
  data["PERF_"+i] = data[i]
  del data[i]

perf_col = ['PERF_BeatenMargin','PERF_FinishPosition','PERF_FinishPosition2','PERF_Disqualified', 'PERF_PIRPosition', 'PERF_Prizemoney', 'PERF_RaceOverallTime', 'PERF_PriceSP', 'PERF_NoFrontCover', 'PERF_PositionInRunning',
            'PERF_WideOffRail']

data = data.reset_index(drop=True)

## Class Restriction: Type and Prize Money Limit

In [14]:
# Cleaning this feature to create two new features, class type and the prize limit to enter the race
data['ClassRestriction_Type'] = data['ClassRestriction'].str.split(' ',expand=True)[0].str.split('$',expand=True)[0]
data['ClassRestriction_Type'] = np.where(((data['ClassRestriction_Type'].isna())|(data['ClassRestriction_Type']=='')),data['ClassRestriction'].str.split(' ',expand=True)[1].str.split('$',expand=True)[0],data['ClassRestriction_Type'])
data['ClassRestriction_Type'] = np.where(((data['ClassRestriction_Type'].isna())|(data['ClassRestriction_Type']=='')),data['ClassRestriction'].str.split(' ',expand=True)[2],data['ClassRestriction_Type'])

data['ClassRestriction_Price'] = data['ClassRestriction'].str.split(' ',expand=True)[0].str.split('$',expand=True)[1]
data['ClassRestriction_Price'] = np.where((data['ClassRestriction_Price'].isna())|(data['ClassRestriction_Price']==''),data['ClassRestriction'].str.split(' ',expand=True)[1].str.split('$',expand=True)[1],data['ClassRestriction_Price'])
data['ClassRestriction_Price'] = np.where((data['ClassRestriction_Price'].isna())|(data['ClassRestriction_Price']==''),0,data['ClassRestriction_Price'])

del data['ClassRestriction']
del data['GoingID']

## Label Encoding

In [15]:
data['Gender'] = np.where(data['Gender']=='F',0,1)

data['RacingSubType'] = np.where(data['RacingSubType']=='TM',1,0)

data['StartType'] = np.where(data['StartType']=='V',1,0)

data['PERF_Disqualified'] = np.where(data['PERF_Disqualified']==False,0,1)

## Foaling Date

In [16]:
data['FoalingDate'] = pd.to_datetime(data['FoalingDate'])
data['RaceStartTime'] = pd.to_datetime(data['RaceStartTime'])

# calculating the age of the horse at the time of the race in days
data['FoalingDate'] = (data['RaceStartTime']-data['FoalingDate']).dt.days

## One Hot Encoding

In [17]:
Categorical_Features = ['AgeRestriction','ClassRestriction_Type','CourseIndicator','FoalingCountry','GoingAbbrev','HandicapType','RaceGroup','Surface','SexRestriction']

Numerical_Features = ['Barrier','ClassRestriction_Price','Distance','FoalingDate','FrontShoes','HandicapDistance','HindShoes','HorseAge','RacePrizemoney','Saddlecloth','StartType',
                      'StartingLine','WeightCarried','WetnessScale','PERF_BeatenMargin','PERF_FinishPosition','PERF_FinishPosition2','PERF_Disqualified','PERF_PIRPosition','PERF_Prizemoney','PERF_RaceOverallTime','PERF_PriceSP','PERF_NoFrontCover',
                      'PERF_PositionInRunning','PERF_WideOffRail','StartType','RacingSubType']

enc = OneHotEncoder()

for i in Categorical_Features:
    # Applying fit_transform, and saving the arrays in a dataframe
    a = pd.DataFrame(enc.fit_transform(data[[i]]).toarray())
    # Mapping each column's names
    a.columns = enc.get_feature_names_out()
    #Joining to original dataframe
    data = data.join(a)
    #Removing original feature
    del data[i]

data = data.reset_index(drop=True)

for i in Numerical_Features:
  data[i] = data[i].astype(float)

data = data.reset_index(drop=True)

## Feature Engineering past performance metrics of Dam, Jockey, Trainer, Sire

In [18]:
# Calculating number of races in each horse
data = data.merge(data.groupby(by=['RaceID'],as_index=False).count().rename(columns={'Barrier':'RaceHorseCount'})[['RaceID','RaceHorseCount']],how='left',on=['RaceID'])

# Calculating Dam's Total Children and Childern's Total Races
data = data.merge(data.groupby(by=['DamID'],as_index=False).agg({ "HorseID": pd.Series.nunique, "Barrier": pd.Series.count}).rename(columns={'HorseID':'Dam_TotalChild','Barrier':'Dam_TotalChildRaces'}),how='left',on=['DamID'])

# Calculating Jockey's Total Races, Total Horses Riden and Mean Beaten Margin
data = data.merge(data.groupby(by=['JockeyID'],as_index=False).agg({"RaceID": pd.Series.nunique,"HorseID": pd.Series.nunique,"PERF_BeatenMargin": pd.Series.mean}).rename(columns={'RaceID':'Jockey_TotalRaces','HorseID':'Jockey_TotalHorses','PERF_BeatenMargin':'Jockey_MeanMargin'}),how='left',on=['JockeyID'])

# Calculating Trainer's Total Races, Total Horses Trained and Mean Beaten Margin
data = data.merge(data.groupby(by=['TrainerID'],as_index=False).agg({"RaceID": pd.Series.nunique,"HorseID": pd.Series.nunique,"PERF_BeatenMargin": pd.Series.mean}).rename(columns={'RaceID':'Trainer_TotalRaces','HorseID':'Trainer_TotalHorses','PERF_BeatenMargin':'Trainer_MeanMargin'}),how='left',on=['TrainerID'])

# Calculating Sire's Total Children and Childern's Total Races
data = data.merge(data.groupby(by=['SireID'],as_index=False).agg({ "HorseID": pd.Series.nunique, "Barrier": pd.Series.count}).rename(columns={'HorseID':'Sire_TotalChild','Barrier':'Sire_TotalChildRaces'}),how='left',on=['SireID'])

## PAST PERFORMANCE FEATURES

In [19]:
data = data.sort_values(by=['HorseID','RaceStartTime'],ascending=[True,True])
data['HorseID'] = data['HorseID'].astype(str)

# Adding the data of last 3 race's performing metrics by shifting the dataset
data = data.join(data[[i for i in data.columns if i in ['HorseID','PERF_FinishPosition2','RaceStartTime','PERF_BeatenMargin','PERF_Disqualified','PERF_PIRPosition','PERF_RaceOverallTime','PERF_NoFrontCover','PERF_PositionInRunning','PERF_WideOffRail','Distance','HandicapDistance','Surface','WeightCarried','WetnessScale']]].shift(1), rsuffix='_Past1')
data = data.join(data[[i for i in data.columns if i in ['HorseID','PERF_FinishPosition2','RaceStartTime','PERF_BeatenMargin','PERF_Disqualified','PERF_PIRPosition','PERF_RaceOverallTime','PERF_NoFrontCover','PERF_PositionInRunning','PERF_WideOffRail','Distance','HandicapDistance','Surface','WeightCarried','WetnessScale']]].shift(2), rsuffix='_Past2')
data = data.join(data[[i for i in data.columns if i in ['HorseID','PERF_FinishPosition2','RaceStartTime','PERF_BeatenMargin','PERF_Disqualified','PERF_PIRPosition','PERF_RaceOverallTime','PERF_NoFrontCover','PERF_PositionInRunning','PERF_WideOffRail','Distance','HandicapDistance','Surface','WeightCarried','WetnessScale']]].shift(3), rsuffix='_Past3')

# Also calculating the time since the horse's last 3 races
data['RaceStartTime_Past1'] = (data['RaceStartTime']-data['RaceStartTime_Past1']).dt.days
data['RaceStartTime_Past2'] = (data['RaceStartTime']-data['RaceStartTime_Past2']).dt.days
data['RaceStartTime_Past3'] = (data['RaceStartTime']-data['RaceStartTime_Past3']).dt.days

# ONLY KEEP THE DATA OF HORSES THAT HAVE ATLEAST 4 RACES, as the data used for training needs atleast 3 prior races data
data = data[(data['HorseID']==data['HorseID_Past1'])&(data['HorseID']==data['HorseID_Past2'])&(data['HorseID']==data['HorseID_Past3'])]
data = data.reset_index(drop=True)

data = data.sort_values(by=['RaceStartTime','PERF_FinishPosition'],ascending=[True,False])
data = data.reset_index(drop=True)

## Save the Processed Dataset in Pickle format on Google Drive

In [20]:
# Just a sanity check, removes any feature that has only 1 unique value
for i in data.columns:
  if data[i].nunique() < 2:
    del data[i]

# Drop Performance Features that we would not need to evaluate the results
data = data.drop(columns=['PERF_RaceOverallTime','DamID','JockeyID','SireID','TrackID','TrainerID','HorseID_Past1','HorseID_Past2','HorseID_Past3','PERF_Disqualified','PERF_PIRPosition','PERF_Prizemoney','PERF_RaceOverallTime','PERF_PriceSP','PERF_NoFrontCover','PERF_PositionInRunning','PERF_WideOffRail'])

# Save the Data in Pickle format on your drive
# data.to_pickle('/content/drive/My Drive/FrenchTrotProcessedData.pkl')

# Unmount your drive
# drive.flush_and_unmount()

# Scaling the Dataset

In [21]:
# drive.mount('/content/drive')
# data = pd.read_pickle('/content/drive/My Drive/FrenchTrotProcessedData.pkl')
del data['RaceStartTime']
del data['PERF_FinishPosition']
# drive.flush_and_unmount()


# # Use Standard Scaler to scale our features
# data_toscale = data.drop(columns=['HorseID','RaceID','PERF_BeatenMargin'])
#
# # Fit the features
# scaler = StandardScaler().fit(data_toscale)
# # Transform the features based on the fit
# data_toscale = scaler.transform(data_toscale)
# # Join Labels back
# data = data[['RaceID','HorseID','PERF_BeatenMargin']].join(pd.DataFrame(data_toscale,columns=data.drop(columns=['RaceID','HorseID','PERF_BeatenMargin']).columns))
# del data_toscale


# EITHER USE STANDARD SCALER OR NORMALIZER AND COMMENT THE OTHER SECTION

# Use the Normalizer from Keras library
normalizer = tf.keras.layers.experimental.preprocessing.Normalization()

# Train the normalizer on Features
normalizer.adapt(data.drop(columns=['PERF_BeatenMargin','HorseID','RaceID','PERF_FinishPosition2'], axis=1))

# CHOOSE A RACE TO PROCEED

This Race ID will be used to predict the model's output for this race. Any Race ID present in the dataset could be fed here. It is recommended to select a race from a later date in the dataset as it would have more past race samples to train on. Also, if the participating horses have more races in the past, the more data the model would have to train on.

Some samples are provided that can be used to run the notebook:
1642204
1638090
1648771
1656357
1650165

In [22]:
raceid = 1638090

# Only store the data of all the horses and their races that participated in this race and free up your memory!

data = data[data['HorseID'].isin(data[data['RaceID']==raceid]['HorseID'])]
data = data.reset_index(drop=True)

# Split into different dataframes for each horse
data_split = [y for x, y in data.groupby('HorseID')]

# MODEL GRID SEARCH TO FIND THE BEST NETWORK PARAMETERS

NOTE: The below grid search code is commented as it takes significant amount of time to run, the cells below directly run the model on the best parameters and the Output of this Grid Search is attached in Excel Format with this Project.

In [23]:
# drive.mount('/content/drive')

# neurons_layer1 = [10,25,50,100]
# neurons_layer2 = [0,10,25,50,100]
# neurons_layer3 = [0,10,25,50,100]
# epochs = [20,50,100]

# cont = 0
# for e in tqdm(epochs):
#   for n1 in neurons_layer1:
#     for n2 in neurons_layer2:
#       for n3 in neurons_layer3:

#         if (e == 20) and (n1 == 10) and (n2 == 0) and (n3 == 0):
#           cont = 1

#         elif cont == 1:

#           # Functional model using pre-processing layer
#           inputs = tf.keras.Input(shape=151)
#           x = normalizer(inputs)

#           # Input Layer
#           x = tf.keras.layers.Dense(151,activation='relu') (x)

#           # Hidden Layer 1
#           x = tf.keras.layers.Dense(n1,activation='relu') (x)

#           if n2 != 0:
#             # Hidden Layer 2
#             x = tf.keras.layers.Dense(n2,activation='relu') (x)
#             if n3 != 0:
#               # Hidden Layer 3
#               x = tf.keras.layers.Dense(n3,activation='relu') (x)

#           # Output Layer
#           output = tf.keras.layers.Dense(1) (x)

#           model = tf.keras.Model(inputs,output)
#           # model.summary()

#           # Train seperate models for each horse
#           for split_frame in data_split:
#             horseid = split_frame['HorseID'].unique()

#             # Do not train on races that the horse got disqualified in
#             split_frame = split_frame[(split_frame['RaceID']<raceid)&(split_frame['PERF_FinishPosition2']!=1)]

#             X = split_frame.drop(columns=['PERF_BeatenMargin','HorseID','RaceID','PERF_FinishPosition2'], axis=1)
#             y = split_frame[['PERF_BeatenMargin']]

#             model = tf.keras.Model(inputs,output)

#             # Complie the model and optimise on MSE
#             model.compile(loss='mean_squared_error',optimizer='adam')

#             # Fit the training data sample by sample by Batch Size of 1, do not shuffle, as we have sorted the data in chronological order to preserve the effect of recent races while training the model
#             model.fit(X,y,batch_size=1,shuffle=False, epochs=e,verbose=0)

#           data = data[data['RaceID']==raceid]


#           # Once all the models are trained for each horse in the chosen race, we predict their beaten margin
#           final = []
#           for i in data['HorseID'].unique():
#             y_pred = model.predict(data[data['HorseID']==i].drop(columns=['PERF_BeatenMargin','HorseID','RaceID','PERF_FinishPosition2'], axis=1),verbose=0)

#             store = pd.DataFrame(y_pred).join(data[data['HorseID']==i]['PERF_BeatenMargin'].reset_index(drop=True))
#             final.append(store)
#           final = pd.concat(final)

#           final = final.sort_values(by=['PERF_BeatenMargin'],ascending=True)
#           final = final.reset_index(drop=True)
#           final = final.sort_values(by=[0],ascending=True)
#           final = final.reset_index(drop=False)

#           # Print the Epoch, Neurons in Layer 1, 2, 3, Mean deviation in predictes finish position of Top 3 horses and RMSE in Beaten Margin for all horses
#           print(e,n1,n2,n3,"RANK: ",str((final[final['index']<3]['index']-final[final['index']<3].index).mean()),"RMSE: ",np.sqrt(((final[0]-final['PERF_BeatenMargin'])**2).mean()))

# GRID SEARCH OUTPUT

Given that the above Grid Search would take a lot of time to run and find out the best network parameters that best minimise the RMSE, we can run the cell below that would directly use the best parameters and predict the output.

NOTE: THE OUTPUT OF ABOVE LOOP IS ATTACHED WITH THIS PROJECT IN EXCEL FORMAT

# Model Predictions

In [24]:
drive.mount('/content/drive')

#Functional model using pre-processing layer
inputs = tf.keras.Input(shape=151)
x = normalizer(inputs)
x = tf.keras.layers.Dense(151,activation='relu') (x)
x = tf.keras.layers.Dense(10,activation='relu') (x)
x = tf.keras.layers.Dense(10,activation='relu') (x)
x = tf.keras.layers.Dense(10,activation='relu') (x)
output = tf.keras.layers.Dense(1) (x)
model = tf.keras.Model(inputs,output)
model.summary()

for split_frame in tqdm(data_split):
  horseid = split_frame['HorseID'].unique()

  split_frame = split_frame[(split_frame['RaceID']<raceid)&(split_frame['PERF_FinishPosition2']!=1)]

  X = split_frame.drop(columns=['PERF_BeatenMargin','HorseID','RaceID','PERF_FinishPosition2'], axis=1)
  y = split_frame[['PERF_BeatenMargin']]

  model = tf.keras.Model(inputs,output)
  model.compile(loss='mean_squared_error',optimizer='adam')
  model.fit(X,y,batch_size=1,shuffle=False, epochs=20,verbose=0)
  model.save_weights('/content/drive/My Drive/model_weight_'+str(horseid[0])+'.h5')
  del model


data = data[data['RaceID']==raceid]


final = []
for i in data['HorseID'].unique():
  model = tf.keras.Model(inputs,output)
  model.compile(loss='mean_squared_error',optimizer='adam')
  model.load_weights('/content/drive/My Drive/model_weight_'+i+'.h5')

  y_pred = model.predict(data[data['HorseID']==i].drop(columns=['PERF_BeatenMargin','HorseID','RaceID','PERF_FinishPosition2'], axis=1))

  store = pd.DataFrame(y_pred).join(data[data['HorseID']==i]['PERF_BeatenMargin'].reset_index(drop=True))
  final.append(store)
final = pd.concat(final)

final = final.sort_values(by=['PERF_BeatenMargin'],ascending=True)
final = final.reset_index(drop=True)
final = final.sort_values(by=[0],ascending=True)
final = final.reset_index(drop=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 151)]             0         
                                                                 
 normalization (Normalizatio  (None, 151)              303       
 n)                                                              
                                                                 
 dense (Dense)               (None, 151)               22952     
                                                                 
 dense_1 (Dense)             (None, 10)                1520      
                                                                 
 dense_2 (Dense)             (None, 10)                110       
                                                                

100%|██████████| 18/18 [01:26<00:00,  4.82s/it]


1/1 [==============================] - 0s 125ms/step


1/1 [==============================] - 0s 126ms/step


1/1 [==============================] - 0s 80ms/step
RANK:  1 RMSE:  12.531485796809166


# FINAL PREDICTIONS ON YOUR CHOSEN RACE:

In [26]:
print("PREDICTED RANK of the Horse the Finished First using the Model that we just trained above: ",str(final[final['index']==0].index[0]))

PREDICTED RANK of the Horse the Finished First using the Model that we just trained above:  1


In [29]:
print("RMSE in Predicting All Horse's Beaten Margin: ",round(np.sqrt(((final[0]-final['PERF_BeatenMargin'])**2).mean()),2),"m")

RMSE in Predicting All Horse's Beaten Margin:  12.53 m


Note: You can go back to the Race ID selection section, select a different Race ID and the re-run the whole notebook, the code would train the model on the dataset for that race and predict the Output.